In [18]:
import numpy as np
import matplotlib.pyplot as plt 
from find_peak_slope import *
from find_peak_value import *
from simulate import *
from zacLib import diff,difff,awgn,mySmooth,IIRsmooth
import numba
%matplotlib Qt5

In [19]:
@numba.njit()
def findBandInRange3(x,th):
    idmax,idmin = 0,0
    xmax,xmin = x[idmax],x[idmin]
    ploc,vloc = [],[]
    rising = -1
    # for i in range(0,x.size):
    for i,d in enumerate(x):
        # d = x[i] 
        if d > xmax:
            idmax = i        
            xmax = x[idmax]
            while d-x[idmin]>th:
                idmin+=1
            xmin = x[idmin]
            rising = idmin
        if d < xmin:
            idmin = i
            xmin = x[idmin]
            while x[idmax]-d>th:
                idmax+=1   
            xmax = x[idmax]
            if rising>=0:
                t = idmax-1
                while t>rising and xmax-x[t]<th:
                    t-=1
                ploc.append(t)
                vloc.append(i)
                rising = -1
    return ploc,vloc

In [20]:
file = 'siga.rd'
sig,fs = rd_reader(file,1,0)
sp = fft(sig,nextpow2(sig.size,0))
sm = IIRsmooth(sp,5,0.01)
sm.tofile('test.dat')
print(sm.size)
# sm = mySmooth(sp,149,2)
# sig,sp = stairs()
# sm = np.fromfile('test.dat')

524288


In [21]:
x = sm.copy()
th = 0.1
oploc,ovloc = findBandInRange(x,th,0,x.size,0)
ploc,vloc = findBandInRange3(x,th)
print(len(oploc),len(ploc))

182 185


In [22]:
oploc,ovloc = findBandInRange(x,th,0,x.size)
plt.plot(x)
plt.plot(oploc,x[oploc],'rs')
plt.plot(ovloc,x[ovloc],'rd')
plt.plot(ploc,x[ploc],'g^')
plt.plot(vloc,x[vloc],'gv')

In [23]:
%timeit findBandInRange3(x,th)
%timeit findBandInRange(x,th,0,x.size,0)

853 µs ± 8.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
15.1 ms ± 335 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
# plt.figure()
# plt.plot(sp,'.',markersize=0.4)
# plt.plot(sm)
# plt.plot(ploc,sm[ploc],'r^')
# plt.plot(vloc,sm[vloc],'gv')
# plt.show()

In [25]:
# def demo(x,sp,width,slope_th):
    # (sm,dsp,new_peak,new_pits,avg_frq,max_frq,
    #  dev_band,half_left,half_right,half_band) = find_peak_slope(sp,width,slope_th)
     
    # plot_all(sp,sm,dsp,slope_th,new_peak,new_pits,avg_frq,
    #          max_frq,dev_band,half_left,half_right,1,origin=x)

In [26]:
# real data

# find_peak_slope_from_rd('sig0.rd',150,20,1,0)
# find_peak_slope_from_rd('sig1.rd',300,30,1,0)
# find_peak_slope_from_rd('sig2.rd',40,10,1,0)
# find_peak_slope_from_rd('sig3.rd',300,30,1,0)
# find_peak_slope_from_rd('siga.rd',300,30,1,0)

In [27]:
# simulate data

# bell shape
# x,sp = bells(noise=0.3)
# demo(x,sp,200,20)

# square
# x,sp = squares(0.5)
# demo(x,sp,100,20)

#stair
# x,sp = stairs(0.5)
# demo(x,sp,200,30)

# random combination
# x,sp = random(noise=0.7)
# demo(x,sp,100,20)